Fernando José Garavito Ovando 18071

Sebastián Maldonado Arnau 18003

# Laboratorio 2 Detección de SPAM

In [1]:
import pandas as pd
import numpy as np
import nltk, re, contractions
import unicodedata
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

## Parte 1 - Ingeniería de características

### Cargar datasets y unirlos

In [2]:
df1 = pd.read_csv("completeSpamAssassin.csv")
df1.head()

,Unnamed: 0,Body,Label
0,0,\nSave up to 70% on Life Insurance.\nWhy Spend...,1
1,1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
2,2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
3,3,##############################################...,1
4,4,I thought you might like these:\n1) Slim Down ...,1


In [3]:
df1 = df1.drop('Unnamed: 0', axis=1)
print(df1.shape)
df1.head()

(6046, 2)


,Body,Label
0,\nSave up to 70% on Life Insurance.\nWhy Spend...,1
1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
3,##############################################...,1
4,I thought you might like these:\n1) Slim Down ...,1


In [4]:
df2 = pd.read_csv("enronSpamSubset.csv")
df2.head()

,Unnamed: 0.1,Unnamed: 0,Body,Label
0,2469,2469,Subject: stock promo mover : cwtd\n * * * urge...,1
1,5063,5063,Subject: are you listed in major search engine...,1
2,12564,12564,"Subject: important information thu , 30 jun 20...",1
3,2796,2796,Subject: = ? utf - 8 ? q ? bask your life with...,1
4,1468,1468,"Subject: "" bidstogo "" is places to go , things...",1


In [5]:
df2 = df2.drop('Unnamed: 0.1', axis=1)
df2 = df2.drop('Unnamed: 0', axis=1)
print(df2.shape)
df2.head()

(10000, 2)


,Body,Label
0,Subject: stock promo mover : cwtd\n * * * urge...,1
1,Subject: are you listed in major search engine...,1
2,"Subject: important information thu , 30 jun 20...",1
3,Subject: = ? utf - 8 ? q ? bask your life with...,1
4,"Subject: "" bidstogo "" is places to go , things...",1


In [6]:
df = pd.concat([df1,df2])
print(df.shape)
df.head()

(16046, 2)


,Body,Label
0,\nSave up to 70% on Life Insurance.\nWhy Spend...,1
1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
3,##############################################...,1
4,I thought you might like these:\n1) Slim Down ...,1


### Preprocesamiento de texto

In [7]:
df['Body']

0       \nSave up to 70% on Life Insurance.\nWhy Spend...
1       1) Fight The Risk of Cancer!\nhttp://www.adcli...
2       1) Fight The Risk of Cancer!\nhttp://www.adcli...
3       ##############################################...
4       I thought you might like these:\n1) Slim Down ...
                              ...                        
9995    Subject: monday 22 nd oct\n louise ,\n do you ...
9996    Subject: missing bloomberg deals\n stephanie -...
9997    Subject: eops salary survey questionnaire\n we...
9998    Subject: q 3 comparison\n hi louise ,\n i have...
9999    Subject: confidential folder to safely pass in...
Name: Body, Length: 16046, dtype: object

In [8]:
stop_words = nltk.corpus.stopwords.words('english')
def procesar(doc):
    doc = str(doc)
    doc = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)','url',doc)
    doc = doc.lower()
    doc = contractions.fix(doc)
    doc = unicodedata.normalize('NFKD', doc).encode('ascii','ignore').decode('utf-8','ignore')
    doc = re.sub(r'[^a-z\s]','',doc)
    tokens = nltk.word_tokenize(doc)
    filtered = [token for token in tokens if token not in stop_words]
    filtered = filtered if len(filtered) < 50 else filtered[:49]
    doc = ' '.join(filtered)
    doc = re.sub(' +', ' ', doc)
    return doc
    

In [9]:
df['Body'] = df['Body'].apply(procesar)
df.sample(10)

,Body,Label
725,ever wondered hell petite asian girls tiny lit...,1
4554,tue donncha caoimh wrote ar mhaith le einne an...,0
3724,url url date mon sep gmtny times piece reporte...,0
746,subject superior prescripiton medicine shattuc...,1
4821,fri aug pm padraig brady wrote kenn humborg wr...,0
4404,subject custom warez cds introduction sell bac...,1
5961,receiving email signed receive one free report...,0
2188,looks sounds hell lot like clares cat violence...,0
6820,subject recovery plan would funny quite close ...,0
5285,cheers adamperhaps people starting notice prof...,0


### Modelos BoW y TF-IDF

Nota: Estos modelos juntos usan aproximadamente 4GB de RAM, si esto es mucha memoria para la computadora en la que se esta corriendo cambiar el min_df de ambos modelos a 0.01, con lo cual usa 0.5GB de RAM

In [10]:
body = np.array(df['Body'])

#### BoW (n=1,2)

In [11]:
cv = CountVectorizer(min_df=0.0005, ngram_range=(1,2))
cv_matrix = cv.fit_transform(body)
cv_matrix = cv_matrix.toarray()
vocabulario = cv.get_feature_names_out()
vocabulario.shape
bow = pd.DataFrame(cv_matrix, columns=vocabulario)
bow

,aa,aa powerfula,aaa,aaaaaa,aactive,aactive color,aactive textdecoration,aall,aall credit,aaron,...,zimbabwe,zimin,zimin lu,zip,zoloft,zone,zonedubai,zonedubai uae,zyban,zyban prozac
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16041,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16042,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16043,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16044,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### TF-IDF

In [12]:
tv = TfidfVectorizer(min_df=0.0003, max_df=1., use_idf=True)
tv_matrix = tv.fit_transform(body)
tv_matrix = tv_matrix.toarray()

vocabulario = tv.get_feature_names_out()
tfidf = pd.DataFrame(np.round(tv_matrix, 2), columns=vocabulario)
tfidf

,aa,aaa,aaaa,aaaaaa,aaaaaaaaaaaa,aactive,aall,aaron,ab,abacha,...,zone,zonedubai,zope,ztop,zu,zy,zyban,zzzz,zzzzspamassassintaintorg,zzzzteana
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.15,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16041,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
16042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
16043,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
16044,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0


## Parte 2 - Implementación